In [46]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

In [47]:
# hàm lấy dữ liệu
def crawl_data_vnexpress(url):
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # tìm tất cả trang báo có trong link
        articles = soup.find_all('article', class_= 'item-news')

        # tạo data
        data = []

        for index, article in enumerate(articles, start=1):

            # lấy title của các trang báo có trong categories đó
            title_tag = article.find('h3', class_= 'title-news') # inspect từ vnexpress
            # nếu tìm được title của bài báo
            if title_tag:
                title = title_tag.get_text(strip=True) # loại dấu thừa

                # lấy link của tất cả bài
                link_tag = title_tag.find('a')
                if link_tag and 'href' in link_tag.attrs:
                    link = link_tag['href']

                    time.sleep(2) # tránh request liên tục bị chặn
                    # truy cập vào link bài báo
                    response_article = requests.get(link, headers={"User-Agent": "Mozilla/5.0"})
                    if response_article. status_code == 200:
                        article_soup = BeautifulSoup(response_article.text, 'html.parser')

                        # lấy tiêu đề phụ
                        sub_title = article_soup.find('p', class_= 'description').get_text(strip=True)
                        # lấy nội dung bài báo
                        content = article_soup.find_all('p', class_= 'Normal')
                        text = '\n'.join([t.get_text(strip=True) for t in content[:-1]]) if content else "None"

                        # lấy tên tác giả
                        author = content[-1].get_text(strip=True) if content else "None"

                        # lưu data vào mảng
                        text = text.strip()
                        data.append([title, link, sub_title, text, author, 'real'])
    return data

def crawl_data_kenh14(url):
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        articles = soup.find_all('h3', class_= ['ktncli-title', 'knswli-title'])
        data = []

        for index, artical in enumerate(articles, start=1):
            link_tag = artical.find('a')

            if link_tag and 'href' in link_tag.attrs:

                # lấy link
                link = "https://kenh14.vn" + link_tag['href']

                time.sleep(2)

                # truy cập bài báo
                response_article = requests.get(link, headers={"User-Agent": "Mozilla/5.0"})

                if response_article.status_code == 200:
                    article_soup = BeautifulSoup(response_article.text, 'html.parser')

                    # lấy title
                    title_tag = article_soup.find('h1', class_='kbwc-title')
                    title = title_tag.get_text(strip=True) if title_tag else "No Title"

                    # lấy sub_title
                    sub_title_tag = article_soup.find('h2', class_='knc-sapo')
                    sub_title = sub_title_tag.get_text(strip=True) if sub_title_tag else "No sub_title"

                    # lấy nội dung
                    content_tag = article_soup.find('div', class_='detail-content')
                    # Check if content_tag is found before proceeding
                    if content_tag:
                        content = content_tag.find_all('p')
                        text = "\n".join([t.get_text(strip=True) for t in content]) if content else "No Content"
                    else:
                        text = "No Content" # Handle case where content_tag is not found

                    # lấy tác giả
                    # Check if the 'kbwc-meta' div exists before searching for the author span
                    author_meta_tag = article_soup.find('div', class_='kbwc-meta')
                    if author_meta_tag:
                        author_tag = author_meta_tag.find('span', class_='kbwcm-author')
                        author = author_tag.get_text().strip(' ,') if author_tag else "Unknown Author"
                    else:
                        author = "Unknown Author"  # Handle case where author information is not found

                    # lưu data
                    text = text.strip()
                    data.append([title, link, sub_title, text, author, 'fake'])
    return data

In [48]:
# ket noi voi drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
# ham luu du lieu cao tu bai bao ve drive file csv
import os

def store_data(data, file_path):
    df_new = pd.DataFrame(data, columns=["Title", "URL", "sub_title", "Content", "Author", "Tag"])

    # kiem tra file da ton tai chua
    if os.path.exists(file_path):
      # new file ton tai roi thi doc dataframe cua file
      df_existing = pd.read_csv(file_path)
      # gop df voi df_existing
      df_combine = pd.concat([df_existing, df_new])
      df_combine = df_combine.drop_duplicates(subset="URL", keep="first")

      # shuffle data
      df_combine = df_combine.sample(frac=1).reset_index(drop=True)

      # luu vao lai file csv
      df_combine.to_csv(file_path, index=False, encoding="utf-8-sig")
      print(f"Save {len(df_new)} line of data to csv successfully")
      print(f"Total lines of data: {len(df_combine)}")
    else:
      # neu file ko ton tai thi tao moi r luu lai
      df_new = df_new.sample(frac=1).reset_index(drop=True)
      df_new.to_csv(file_path, index=False, encoding="utf-8-sig")
      print(f"Save {len(df_new)} line of data to csv successfully")

In [49]:
# ham lay categories cua vnexpress
def vnexpress_categories(path):
  response = requests.get(path, headers={"User-Agent": "Mozilla/5.0"})
  if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      print("Connect successfully")

      categories = soup.find('ul', class_='parent')
      categories = categories.find_all('a')

      # bo 2 phan tu dau va phan tu cuoi, bo luon muc dan den video vi khong can thiet
      categories = [c['href'] for c in categories[2:-4] if c['href'] != 'https://video.vnexpress.net']
      print(categories)
      return categories

In [51]:
# path toi drive
file_path = "/content/drive/MyDrive/NLP/mid_term/data.csv"

In [52]:
# lay categories cua vnexpress
url_vnexpress = "https://vnexpress.net" # base url
categories_vnexpress = vnexpress_categories(url_vnexpress)

Connect successfully
['/thoi-su', '/the-gioi', '/kinh-doanh', '/cong-nghe', '/khoa-hoc', '/podcast', '/goc-nhin', '/bat-dong-san', '/suc-khoe', '/the-thao', '/giai-tri', '/phap-luat', '/giao-duc', '/doi-song', '/oto-xe-may']


In [21]:
# cao data vnexpress
for category in categories_vnexpress:
  print(f"Dang lay du lieu tu {category} cua vnexpress")
  full_url = url_vnexpress + category
  data_vnexpress = crawl_data_vnexpress(full_url)

  # lay duoc 1 category thi luu vao file csv
  store_data(data_vnexpress, file_path)
  print("--------------------- \n")

Dang lay du lieu tu /thoi-su cua vnexpress
Save 44 line of data to csv successfully
Total lines of data: 616
--------------------- 

Dang lay du lieu tu /the-gioi cua vnexpress
Save 44 line of data to csv successfully
Total lines of data: 618
--------------------- 

Dang lay du lieu tu /kinh-doanh cua vnexpress
Save 47 line of data to csv successfully
Total lines of data: 620
--------------------- 

Dang lay du lieu tu /cong-nghe cua vnexpress
Save 43 line of data to csv successfully
Total lines of data: 621
--------------------- 

Dang lay du lieu tu /khoa-hoc cua vnexpress
Save 35 line of data to csv successfully
Total lines of data: 624
--------------------- 

Dang lay du lieu tu /podcast cua vnexpress
Save 5 line of data to csv successfully
Total lines of data: 625
--------------------- 

Dang lay du lieu tu /goc-nhin cua vnexpress
Save 19 line of data to csv successfully
Total lines of data: 625
--------------------- 

Dang lay du lieu tu /bat-dong-san cua vnexpress
Save 62 line o

# Cào VNExpress tới du lịch thì lỗi nhưng được hơn 600 dòng nên dừng lại cào tới trang kenh14
=> cào khoảng 15 categories thoi ahihi

In [20]:
# lấy categories của kenh14 cào cho nhiều bài

url_kenh14 = "https://kenh14.vn"

# hàm lấy categories của kenh14
def kenh14_categories(path):
  response = requests.get(path, headers={"User-Agent": "Mozilla/5.0"})
  if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')
      print("Connect successfully")

      categories = soup.find('ul', class_='kbh-menu-list')
      categories = categories.find_all('a')

      categories = [c['href'] for c in categories[1:18] if c['href'] not in ['http://video.kenh14.vn/', 'javascript:;']]
      return categories

categories_kenh14 = kenh14_categories(url_kenh14)
print(categories_kenh14)

Connect successfully
['/star.chn', '/cine.chn', '/musik.chn', '/beauty-fashion.chn', '/doi-song.chn', '/money-z.chn', '/an-quay-di.chn', '/xa-hoi.chn', '/suc-khoe.chn', '/tek-life.chn', '/hoc-duong.chn', '/xem-mua-luon.chn', '/doi-song.chn', '/doi-song/nhan-vat.chn', '/xem-an-choi.chn']


In [26]:
# cao data kenh14
for category in categories_kenh14:
  print(f"Dang lay du lieu tu {category} cua kenh14")
  full_url = url_kenh14 + category
  data_kenh14 = crawl_data_kenh14(full_url)

  # lay duoc 1 category thi luu vao file csv
  store_data(data_kenh14, file_path)

Dang lay du lieu tu /star.chn cua kenh14
Save 10 line of data to csv successfully
Total lines of data: 741
Dang lay du lieu tu /cine.chn cua kenh14
Save 10 line of data to csv successfully
Total lines of data: 741
Dang lay du lieu tu /musik.chn cua kenh14
Save 10 line of data to csv successfully
Total lines of data: 741
Dang lay du lieu tu /beauty-fashion.chn cua kenh14
Save 7 line of data to csv successfully
Total lines of data: 741
Dang lay du lieu tu /doi-song.chn cua kenh14
Save 10 line of data to csv successfully
Total lines of data: 741
Dang lay du lieu tu /money-z.chn cua kenh14
Save 10 line of data to csv successfully
Total lines of data: 742
Dang lay du lieu tu /an-quay-di.chn cua kenh14
Save 1 line of data to csv successfully
Total lines of data: 742
Dang lay du lieu tu /xa-hoi.chn cua kenh14
Save 10 line of data to csv successfully
Total lines of data: 742
Dang lay du lieu tu /suc-khoe.chn cua kenh14
Save 10 line of data to csv successfully
Total lines of data: 742
Dang lay 

In [44]:
# cào thêm 1 trang báo làm fake news
# báo znews
# hàm lấy categories của trang
def znews_categories(path):
  response = requests.get(path, headers={"User-Agent": "Mozilla/5.0"})
  if response.status_code == 200:
      soup = BeautifulSoup(response.text, 'html.parser')

      # lấy tag category
      category_tag = soup.find('nav', class_= 'category-menu')
      link_tag = category_tag.find_all('a')
      link = [l['href'] for l in link_tag]
      return link

In [45]:
znews_links = znews_categories("https://znews.vn")
print(znews_links)

['https://znews.vn/xuat-ban.html', 'https://znews.vn/tac-gia.html', 'https://znews.vn/kinh-doanh-tai-chinh.html', 'https://znews.vn/the-thao.html', 'https://znews.vn/cong-nghe.html', 'https://lifestyle.znews.vn/suc-khoe.html', 'https://lifestyle.znews.vn/doi-song.html', 'https://znews.vn/giai-tri.html']


In [79]:
# hàm crawl dữ liệu trang znews
def crawl_data_znews(url):
  response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
  if response.status_code == 200:

    soup = BeautifulSoup(response.text, 'html.parser')

    # lấy tất cả bài trong trang
    articles = soup.find_all('p', class_= 'article-title')

    data = []
    for article in articles:
      link_tag = article.find('a')
      if link_tag and 'href' in link_tag.attrs:
        link = link_tag['href']

        # title
        title = link_tag.get_text(strip=True)
        time.sleep(2)

        if link:
          response_article = requests.get(link, headers={"User-Agent": "Mozilla/5.0"})
          if response_article.status_code == 200:
            article_soup = BeautifulSoup(response_article.text, 'html.parser')

            # lấy sub_title
            sub_title_tag = article_soup.find('p', class_='the-article-summary')
            sub_title = sub_title_tag.get_text(strip=True) if sub_title_tag else "No sub_title"

            # lấy content
            content_tag = article_soup.find('div', class_='the-article-body')
            content = content_tag.find_all('p') if content_tag else None
            text = "\n".join([t.get_text(strip=True) for t in content]) if content else "No Content"

            # tác giả
            author_tag = article_soup.find('li', class_='the-article-author')
            author = author_tag.get_text(strip=True) if author_tag else "Unknown Author"

            text = text.strip()
            data.append([title, link, sub_title, text, author, 'fake'])

    return data

In [86]:
# crawl data từ znews

for link in znews_links:
  print(f"Dang lay du lieu tu {link} cua znews")
  data_znews = crawl_data_znews(link)

  # lay duoc 1 category thi luu vao file csv
  store_data(data_znews, file_path)
  print("--------------------- \n")

Dang lay du lieu tu https://znews.vn/xuat-ban.html cua znews
Save 50 line of data to csv successfully
Total lines of data: 812
--------------------- 

Dang lay du lieu tu https://znews.vn/tac-gia.html cua znews
Save 50 line of data to csv successfully
Total lines of data: 860
--------------------- 

Dang lay du lieu tu https://znews.vn/kinh-doanh-tai-chinh.html cua znews
Save 67 line of data to csv successfully
Total lines of data: 925
--------------------- 

Dang lay du lieu tu https://znews.vn/the-thao.html cua znews
Save 48 line of data to csv successfully
Total lines of data: 973
--------------------- 

Dang lay du lieu tu https://znews.vn/cong-nghe.html cua znews
Save 53 line of data to csv successfully
Total lines of data: 1026
--------------------- 

Dang lay du lieu tu https://lifestyle.znews.vn/suc-khoe.html cua znews
Save 71 line of data to csv successfully
Total lines of data: 1097
--------------------- 

Dang lay du lieu tu https://lifestyle.znews.vn/doi-song.html cua znews